In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
puzzle_data = pd.read_csv("../full_meta_clustered_singletag.tsv", sep='\t')

In [2]:
#Load the full dataset of puzzle attempts
attempts = pd.read_csv("/w/225/1/chess/tactics/glicko_user_tactics_problem.csv_00")

<h2>Get Average User Success Sample Var in Actual Clusters</h2>

In [9]:
#Join cluster labels to puzzles that were clustered 
attempts=pd.merge(attempts,puzzle_data[['tactics_problem_id','cluster']],on='tactics_problem_id', how='inner')

In [3]:
#Filter attempts to include only attempts by players who have played at least 100 games
attempts = attempts.loc[attempts['userGamesPlayed'] >= 100]

In [129]:
attempts.head()

,user_tactics_problem_id,user_hash,create_date,date,last_game_date,seconds,is_passed,correct_move_count,rating_change,tactics_problem_id,ratingUser,varianceUser,ratingProblem,varianceProblem,userGamesPlayed,success,cluster
0,2822621,ec2164fc960c33cb378fb589ec23a455fac6cd575c62ab...,2008-04-02 15:58:44,2008-04-02,2008-04-02,10,0,0,-27,27060,1600,75,1928,261,100,0.131459,9
1,2831172,cd1950759a710929d8a76f9e531f0d18de64c8de6141ad...,2008-04-03 06:15:38,2008-04-03,2008-04-03,14,0,0,-21,27060,1773,44,1717,134,178,0.579900,9
2,2833192,79be8658df80df08ab425b1c20ba41770de5cee5949a36...,2008-04-03 09:18:25,2008-04-03,2008-04-03,5,1,1,53,27060,1807,38,1724,118,296,0.617225,9
3,2835324,e8f281ea2931e4e2c562c774aa1ec72b5ab88ffb789e9d...,2008-04-03 12:01:48,2008-04-03,2008-04-03,19,1,1,11,27060,1773,35,1696,112,193,0.609033,9
4,2849652,0575612356f922da19918aa0b69d42a19886aec9d5aa71...,2008-04-04 12:23:27,2008-04-04,2008-04-04,16,0,0,-22,27060,1818,44,1720,85,275,0.637408,9


In [6]:
#Add expected success for each player using formula for ELO expected score here: 
#https://en.wikipedia.org/wiki/Elo_rating_system#Mathematical_details

def add_success(row):
    prob_rating = row['ratingProblem']
    rating_user = row['ratingUser']
    rating_diff = prob_rating - rating_user
    return 1/(1 + 10**(rating_diff/400))

attempts['success'] = attempts.progress_apply(lambda row: add_success(row),axis=1)

In [25]:
#Get average user success for each cluster
user_cluster_averaged = attempts.groupby(['user_hash','cluster'])['success'].agg('mean')
#Get the variance of average cluster success and then take the average
avg_cluster_variance = user_cluster_averaged.var(level=0).agg('mean')
print(avg_cluster_variance)

0.013883403671947049


<h2>Get Average User Success Sample Var in Fake Clusters</h2>

In [93]:
#Get number of puzzles in each cluster and create a numpy array of cluster labels so that the number of labels
#for cluster 'a' correspond to the count of cluster a in the original puzzle metadata
cluster_count = puzzle_data.groupby('cluster')['tactics_problem_id'].nunique().values
cluster_list = np.array([])
for i in range(20):
    clust_labels = np.full(cluster_count[i],i+1)
    cluster_list = np.append(cluster_list,clust_labels)
cluster_list = cluster_list.astype(int)

In [117]:
#Shuffle the order of the rows of the original puzzle metadata
puzzle_data_shuffled = puzzle_data.copy().sample(frac=1).reset_index(drop=True)
#Add the clusters to the shuffled puzzles so that the mismatch will create clusters of equal size as the original
#But with the puzzles grouped randomly
puzzle_data_shuffled['cluster'] = cluster_list
#Shuffle again for appearance
puzzle_data_shuffled = puzzle_data_shuffled.sample(frac=1).reset_index(drop=True)

In [118]:
puzzle_data_shuffled.head()

,tactics_problem_id,rating,rd,attempt_count,average_seconds,move_count,tags,fen,tag,cluster
0,70491,400,40.15,4248,28,1,Mate in 1,https://lichess.org/analysis/r7/p2R4/1p3pk1/2p...,Mate in 1,14
1,66730,1617,45.20,49296,57,4,"Discovered Attack,Mate in 3+,Pin,Skewer,Vulner...",https://lichess.org/analysis/5rk1/pp2qrp1/2p1p...,Skewer,8
2,155014,1163,45.12,62286,27,2,"Attacking Castled King,Basic Checkmates,Discov...",https://lichess.org/analysis/1n1r1r1k/pppb1ppB...,Interference,8
3,82146,1385,45.41,61340,29,1,"Mate in 2,Mating Net,Vulnerable King",https://lichess.org/analysis/3q1k1r/1p2b2P/r2p...,Vulnerable King,14
4,46309,811,43.99,48267,48,1,"Fork / Double Attack,Pin",https://lichess.org/analysis/r1bn1rk1/3n2p1/1p...,Pin,8


In [121]:
attempts.drop(['cluster'],axis=1,inplace=True)
attempts=pd.merge(attempts,
                  puzzle_data_shuffled[['tactics_problem_id','cluster']],
                  on='tactics_problem_id', 
                  how='inner')

In [122]:
#Get average user success for each cluster
user_cluster_averaged = attempts.groupby(['user_hash','cluster'])['success'].agg('mean')
#Get the variance of average cluster success and then take the average
avg_cluster_variance = user_cluster_averaged.var(level=0).agg('mean')
print(avg_cluster_variance)

0.007632470212113265


In [124]:
#Loop that recreates random baseline clusters and recomputes the average variance of player skills in each cluster
results = []
for i in tqdm(range(10)):
    puzzle_data_shuffled = puzzle_data.copy().sample(frac=1).reset_index(drop=True)
    puzzle_data_shuffled['cluster'] = cluster_list
    attempts.drop(['cluster'],axis=1,inplace=True)
    attempts=pd.merge(attempts,
                      puzzle_data_shuffled[['tactics_problem_id','cluster']],
                      on='tactics_problem_id', 
                      how='inner')
    #Get average user success for each cluster
    user_cluster_averaged = attempts.groupby(['user_hash','cluster'])['success'].agg('mean')
    #Get the variance of average cluster success and then take the average
    avg_cluster_variance = user_cluster_averaged.var(level=0).agg('mean')
    results.append(avg_cluster_variance)

In [128]:
print(results)

[0.007747881945537818, 0.00741636836770366, 0.00793006632924327, 0.007580549637144724, 0.0072726002652239, 0.007482436331059905, 0.007156147995837817, 0.007554701906236195, 0.007303807171266644, 0.007404616147067197]


In [127]:
print(min(results))
print(sum(results)/10)
print(max(results))

0.007156147995837817
0.007484917609632114
0.00793006632924327
